In [ ]:
%qtconsole

In [ ]:
import numpy as np
import pandas as pd
import scipy.signal as signal
from scipy.fftpack import fft, ifft, fftfreq
import sys
sys.path.insert(0,'..')
from levitate.nonlinear import hammersteinModel
from levitate.parametricaudio import ParametricAudioModel
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot
output_notebook()
TOOLS = 'pan,box_zoom,wheel_zoom,reset'

In [ ]:
noiselevel = 0.5
addnoise = lambda x: x + (2*np.random.rand(len(x))-1)*noiselevel*np.max(np.abs(x))
n=2**14
fs=192e3
f0=1e3
t=np.arange(n)/fs


lpf = signal.iirfilter(24,24e3/(fs/2),btype='low')

paramAudio = ParametricAudioModel(
    modulation='simple',demodulation='detection',
    fs=fs, padcycles=100,windowcycles=400,
    equalize='no',lpf=False)

inputsignal = 2*np.random.rand(n)-1
inputsignal = signal.lfilter(lpf[0], lpf[1], inputsignal)

#inputsignal = signal.chirp(t,20,t[-1],24e3)
#inputsignal = np.sin(2*np.pi*f0*t)
inputsignal /= np.max(np.abs(inputsignal))
timedata = pd.DataFrame({'input':inputsignal,'t':t})
timedata['noise free output'] = paramAudio(inputsignal)
timedata['output'] = addnoise(timedata['noise free output'])
timedata['output'] /= np.max(np.abs(timedata['output']))
timedata['window'] = paramAudio.window

smoothing = np.ceil(paramAudio.windowcycles*paramAudio.fs/paramAudio.fc).astype('int')
#inputsignal=inputsignal[smoothing:-smoothing]
#outputsignal=outputsignal[smoothing:-smoothing]
timedata.drop(timedata.head(smoothing).index,inplace=True)
timedata.drop(timedata.tail(smoothing).index,inplace=True)

freqdata = pd.DataFrame({
    'f': fftfreq(len(timedata),1/fs),
    'input': fft(timedata['input']),
    'output': fft(timedata['output'])
})

nonlinearity = pd.DataFrame({'x':np.linspace(-1,1,1e3)})
linearity = pd.DataFrame({'t': np.arange(rectangular.irlen)/fs})

rectangular = hammersteinModel(kernel='rectangular',npoints=500)
rectangular.approximateNonlinearity(timedata['input'],timedata['output'])
nonlinearity['rectangular'] = rectangular.nonlinearity(nonlinearity['x'])
rectangular.approximateLinearity(timedata['input'],timedata['output'])
linearity['rectangular'] = rectangular.impulseResponse

#sincsq = hammersteinModel(kernel='sincsq',npoints=500)
#sincsq.approximateNonlinearity(timedata['input'],timedata['output'])
#nonlinearity['sincsq'] = sincsq.nonlinearity(nonlinearity['x'])
#sincsq.approximateLinearity(timedata['input'],timedata['output'])
#linearity['sincsq'] = sincsq.impulseResponse

triangular = hammersteinModel(kernel='triangular',npoints=500)
triangular.approximateNonlinearity(timedata['input'],timedata['output'])
nonlinearity['triangular'] = triangular.nonlinearity(nonlinearity['x'])
triangular.approximateLinearity(timedata['input'],timedata['output'])
linearity['triangular'] = triangular.impulseResponse

gauss = hammersteinModel(kernel='gauss',npoints=500)
gauss.approximateNonlinearity(timedata['input'],timedata['output'])
nonlinearity['gauss'] = gauss.nonlinearity(nonlinearity['x'])
gauss.approximateLinearity(timedata['input'],timedata['output'])
linearity['gauss'] = gauss.impulseResponse

inputsignal = 2*np.random.rand(n)-1
lpf = signal.iirfilter(24,24e3/(fs/2),btype='low')
inputsignal = signal.lfilter(lpf[0], lpf[1], inputsignal)
inputsignal /= np.max(np.abs(inputsignal))
outputsignal = addnoise(paramAudio(inputsignal))
outputsignal /= np.max(np.abs(outputsignal))

gauss.refineModel(inputsignal,outputsignal)
nonlinearity['refined gauss'] = gauss.nonlinearity(nonlinearity['x'])
linearity['refined gauss'] = gauss.impulseResponse


In [ ]:
f1=figure(tools=TOOLS+',box_select,lasso_select',title='Input')
f2=figure(tools=TOOLS+',box_select,lasso_select',title='Output')
f3=figure(tools=TOOLS+',box_select,lasso_select',x_axis_label='Input',y_axis_label='Output')
cds=ColumnDataSource(timedata)
f1.line(x='t',y='input',source=cds,selection_color='orange',nonselection_alpha=0.5)
f1.circle(x='t',y='input',source=cds,alpha=0,nonselection_alpha=0)
f2.line(x='t',y='output',source=cds,selection_color='orange',nonselection_alpha=0.5)
f2.circle(x='t',y='output',source=cds,alpha=0,nonselection_alpha=0)
f3.circle(x='input',y='output',source=cds,selection_color='orange')
p=gridplot([[f1],[f2],[f3]])
show(p)

In [ ]:
f1=figure(tools=TOOLS,x_axis_type='log')
f2=figure(tools=TOOLS,x_axis_type='log')
f1.line(freqdata['f'],20*np.log10(np.abs(freqdata['input'])))
f2.line(freqdata['f'],20*np.log10(np.abs(freqdata['output'])))
p=gridplot([[f1],[f2]])
show(p)


In [ ]:
f=figure(tools=TOOLS,title='Nonlinear approximations')
fun = lambda x: -x**2
f.line(nonlinearity['x'],nonlinearity['rectangular'],legend='Rectangular')
f.line(nonlinearity['x'],nonlinearity['triangular'],legend='Triangular',color='orange')
f.line(nonlinearity['x'],nonlinearity['gauss'],legend='Gaussian',color='green')
f.line(nonlinearity['x'],nonlinearity['refined gauss'],legend='Refined Gaussian',color='red')
f.line(nonlinearity['x'],fun(nonlinearity['x']),legend='Analytical',color='black')
show(f)

In [ ]:


f=figure(tools=TOOLS,title='Impulse response')
f.line(linearity['t'],linearity['rectangular'],legend='Rectangular')
f.line(linearity['t'],linearity['gauss'],legend='Gaussian',color='green')
f.line(linearity['t'],linearity['refined gauss'],legend='Refined gaussian',color='red')
show(f)